In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
import datetime

# Load dữ liệu (như code trước)
tbtl_data = pd.read_excel("/kaggle/input/wecode-public-it001/data-tbtl/tbtl-public.ods", engine="odf")
th_data = pd.read_csv("/kaggle/input/wecode-public-it001/public_it001/th-public.csv")
qt_data = pd.read_csv("/kaggle/input/wecode-public-it001/public_it001/qt-public.csv")
anno_data = pd.read_csv("/kaggle/input/wecode-public-it001/data-tbtl/annonimized.csv")
ck_data = pd.read_csv("/kaggle/input/wecode-public-it001/public_it001/ck-public.csv")

anno_data.columns = ['assignment_id','problem_id','username','is_final','status','pre_score','coefficient','language_id','created_at','updated_at','judgement']

In [14]:
def time_diff(df):
    current_year = 2024
    
    # Xử lý 'created_at'
    try:
        df['created_at'] = pd.to_datetime(df['created_at'].astype(str) + '-' + str(current_year), format='%m-%d %H:%M:%S-%Y')
    except (ValueError, TypeError):
        df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

    # Xử lý 'updated_at' (đảm bảo không bị bỏ qua)
    try:
        df['updated_at'] = pd.to_datetime(df['updated_at'].astype(str) + '-' + str(current_year), format='%m-%d %H:%M:%S-%Y')
    except (ValueError, TypeError):
        df['updated_at'] = pd.to_datetime(df['updated_at'], errors='coerce')

    # Tính 'time_diff' giữa 'updated_at' và 'created_at'
    df['time_diff'] = (df['updated_at'] - df['created_at']).dt.total_seconds().abs()
    return df
    
def one_hot_encode(tasks, all_tasks):
    return [1 if task in tasks else 0 for task in all_tasks]
    
def tasks_to_embedding(tasks, embedding_matrix):
        vectors = [embedding_matrix[task - 1] for task in tasks]  # Trừ 1 để phù hợp index
        return np.mean(vectors, axis=0)  # Trung bình các vector

In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def clean_and_aggregate_data(anno_data):
    cols_to_drop = ['language_id', 'created_at', 'updated_at', 'judgement', "is_final", "status"]
    df = anno_data.drop(columns=cols_to_drop)

    le = LabelEncoder()
    df['assignment_id'] = le.fit_transform(df['assignment_id'])

    df["submission_count"] = df.groupby(["assignment_id", "username"])["assignment_id"].transform('size')

    df = df.groupby(["username", "assignment_id"]).agg(
        average_pre_score=("pre_score", "mean"),
        average_coefficient=("coefficient", "mean"),
        submission_count=("submission_count", "max")
    ).reset_index()
    df["average_pre_score"] = df["average_pre_score"]
    df["average_coefficient"] = df["average_coefficient"]
    df = df.groupby('username').agg({
        'assignment_id': lambda x: list(x),
        'average_pre_score': 'mean',
        'average_coefficient': 'mean',
        'submission_count': 'sum'
    }).reset_index()
    df2 = anno_data
    df2 = time_diff(df2)
    
    # Tính số lần submission
    df2["submission_count"] = df2.groupby(["assignment_id", "problem_id", "username"]).cumcount() + 1
    df2['total_time'] = df2.groupby(["assignment_id", "problem_id", "username"])['time_diff'].transform('sum')
    
    # Lọc ra 1 dòng cuối cho mỗi nhóm
    df2 = df2.groupby(["assignment_id", "problem_id", "username"]).tail(1).reset_index(drop=True)
    df2 = df2.groupby("username").agg(
        total_problems=("problem_id", "count"),
        total_time=("total_time", "sum"),
    ).reset_index()
    
    df_merged = df.merge(df2[['username', 'total_problems',"total_time"]], on='username', how='left')

    #thêm đặc trưng
    df_merged["solved_ratio"] = (df_merged["total_problems"] / df_merged["submission_count"])
    df_merged['submission_efficiency'] = df_merged['submission_count'] / df_merged['total_problems']
    df_merged['adjusted_pre_score'] = df_merged['average_pre_score'] * df_merged['average_coefficient']
    df_merged['log_submission_count'] = np.log1p(df_merged['submission_count'])
    df_merged['log_total_problems'] = np.log1p(df_merged['total_problems'])
    df_merged['poly_4_average_pre_score'] = df_merged['average_pre_score'] ** 4
    
    #onehot_encode
    assignment_id_uniques = set(task for tasks in df_merged['assignment_id'] for task in tasks)
    assignment_id_uniques = sorted(list(assignment_id_uniques))

        
    df_merged['assignment_id'] = df_merged['assignment_id'].apply(lambda x: one_hot_encode(x, assignment_id_uniques))

    # tạo embedding
    num_tasks = len(anno_data['assignment_id'].unique())
    embedding_dim = 16
    embedding_matrix = np.random.rand(num_tasks, embedding_dim)
    
    df_merged['embedding'] = df_merged['assignment_id'].apply(lambda x: tasks_to_embedding(x, embedding_matrix))

    #thêm đặc trưng
    df_merged['embedding_mean'] = df_merged['embedding'].apply(lambda x: np.mean(x))
    df_merged['embedding_std'] = df_merged['embedding'].apply(lambda x: np.std(x))
    df_merged['embedding_max'] = df_merged['embedding'].apply(lambda x: np.max(x))
    df_merged['embedding_min'] = df_merged['embedding'].apply(lambda x: np.min(x))
    return df_merged

df = clean_and_aggregate_data(anno_data)
df.head()

,username,assignment_id,average_pre_score,average_coefficient,submission_count,total_problems,total_time,solved_ratio,submission_efficiency,adjusted_pre_score,log_submission_count,log_total_problems,poly_4_average_pre_score,embedding,embedding_mean,embedding_std,embedding_max,embedding_min
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",5838.193956,100.0,147,47,1204728.0,0.319728,3.127660,583819.395604,4.997212,3.871201,1.161754e+15,"[0.4278805397705205, 0.5370963724059248, 0.620...",0.386378,0.218217,0.796809,0.024609
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5473.090254,100.0,259,78,65265092.0,0.301158,3.320513,547309.025449,5.560682,4.369448,8.972851e+14,"[0.4325738747761158, 0.5389242571965164, 0.622...",0.388178,0.216841,0.796064,0.031022
2,01122b3ef7e59b84189e65985305f575d6bdf83c,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6953.009206,100.0,195,68,23636384.0,0.348718,2.867647,695300.920552,5.278115,4.234107,2.337175e+15,"[0.42788053977052054, 0.5370963724059248, 0.62...",0.386378,0.218217,0.796809,0.024609
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6952.644118,100.0,100,47,151409.0,0.470000,2.127660,695264.411765,4.615121,3.871201,2.336684e+15,"[0.42084053726212756, 0.5343545452200371, 0.61...",0.383678,0.220325,0.797928,0.014990
4,013de369c439ab0ead8aa7da64423aa395a8be39,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",6454.259540,87.5,107,66,19692.0,0.616822,1.621212,564747.709741,4.682131,4.204693,1.735344e+15,"[0.43022720727331815, 0.5380103148012206, 0.62...",0.387278,0.217526,0.796437,0.027816


In [16]:
ck_data['CK'] = pd.to_numeric(ck_data['CK'], errors='coerce')
ck_data = ck_data.dropna(subset=['CK'])
print(ck_data['CK'].isna().sum()) 

0


In [17]:
print(f"Total_student = {df.shape[0]}")

anno_student = set(df["username"])
print (f"length anno_student = {len(anno_student)}")

tbtl_student = set(tbtl_data["username"])
print (f"length tbtl_student = {len(tbtl_student)}")

predict_student = anno_student - tbtl_student
print (f"length predict_student = {len(predict_student)}")

Total_student = 1489
length anno_student = 1489
length tbtl_student = 799
length predict_student = 690


In [18]:
predict_df = df[df["username"].isin(predict_student)]

print(f"predict_student = {predict_df.shape[0]}")

predict_df.head()

predict_student = 690


,username,assignment_id,average_pre_score,average_coefficient,submission_count,total_problems,total_time,solved_ratio,submission_efficiency,adjusted_pre_score,log_submission_count,log_total_problems,poly_4_average_pre_score,embedding,embedding_mean,embedding_std,embedding_max,embedding_min
394,410357eb9129023509cfaf8d38be61c050bb3b05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5542.025641,100.0,43,16,12714.0,0.372093,2.687500,554202.564103,3.784190,2.833213,9.433528e+14,"[0.41614720225653223, 0.5325266604294454, 0.61...",0.381878,0.221759,0.798673,0.008577
625,67212308d026508fd5b6942ffbbdd7b0be2e89de,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,100.0,8,2,48.0,0.250000,4.000000,0.000000,2.197225,1.098612,0.000000e+00,"[0.41380053475373463, 0.5316127180341496, 0.61...",0.380978,0.222485,0.799046,0.005371
801,84a17972cc6d29489bbe205a9e7feb8745726fbc,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7742.667691,100.0,182,110,1460062.0,0.604396,1.654545,774266.769073,5.209486,4.709530,3.593871e+15,"[0.4255338722677228, 0.5361824300106289, 0.620...",0.385478,0.218913,0.797182,0.021403
802,84b6b2d70924066c8345f2bc2281791ae3188da2,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",6787.268610,100.0,160,82,62797979.0,0.512500,1.951220,678726.860957,5.081404,4.418841,2.122170e+15,"[0.4278805397705206, 0.5370963724059249, 0.620...",0.386378,0.218217,0.796809,0.024609
803,851d9a4b9b8e236f2d62282ddf06fae57b7d9492,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5277.509215,100.0,389,97,11657734.0,0.249357,4.010309,527750.921516,5.966147,4.584967,7.757397e+14,"[0.43961387728450874, 0.541666084382404, 0.625...",0.390877,0.214824,0.794946,0.040642


In [19]:
df = pd.merge(df, ck_data, left_on='username',right_on='hash', how='inner')

In [33]:
columns_features =['average_pre_score', 'average_coefficient', 'solved_ratio','submission_efficiency',
                   'submission_count','total_problems',
                   'log_submission_count','log_total_problems',
                   'poly_4_average_pre_score',	'embedding_mean',	'embedding_std',	'embedding_max',	'embedding_min']

features = df[columns_features]

X = np.hstack((np.vstack(df['embedding']), features))
y = df['CK']
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X_train[1]

array([-1.46153442, -1.46153442, -1.46153442, -1.46153442,  1.46153442,
        1.46153442, -1.46153442,  1.46153442, -1.46153442,  1.46153442,
       -1.46153442, -1.46153442, -1.46153442, -1.46153442,  1.46153442,
       -1.46153442, -0.44715175,  0.22221727,  0.93327368, -0.87122124,
       -1.35807865, -1.68912927, -1.83995863, -1.74892668, -0.61006671])

# Train bình thường😉

In [34]:
# Xây dựng mô hình mạng học sâu
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
# Biên dịch mô hình
model.compile(optimizer=Adam(learning_rate=0.005), loss='mse')

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=80, batch_size=16, validation_data=(X_test, y_test), verbose=False)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Tính R^2
r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
R^2: 0.10998898980077687


In [35]:
predict_features = predict_df[columns_features]
X_predict = np.hstack((np.vstack(predict_df['embedding']), predict_features))

X_predict_scaled = scaler.fit_transform(X_predict)
y_real_pred = model.predict(X_predict_scaled)

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [36]:
# Assign the rounded values to the predicted_scores column
predict_df["predicted_score"] = np.round(y_real_pred, 3)

predict_df.head()

,username,assignment_id,average_pre_score,average_coefficient,submission_count,total_problems,total_time,solved_ratio,submission_efficiency,adjusted_pre_score,log_submission_count,log_total_problems,poly_4_average_pre_score,embedding,embedding_mean,embedding_std,embedding_max,embedding_min,predicted_score
394,410357eb9129023509cfaf8d38be61c050bb3b05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5542.025641,100.0,43,16,12714.0,0.372093,2.687500,554202.564103,3.784190,2.833213,9.433528e+14,"[0.41614720225653223, 0.5325266604294454, 0.61...",0.381878,0.221759,0.798673,0.008577,4.696
625,67212308d026508fd5b6942ffbbdd7b0be2e89de,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,100.0,8,2,48.0,0.250000,4.000000,0.000000,2.197225,1.098612,0.000000e+00,"[0.41380053475373463, 0.5316127180341496, 0.61...",0.380978,0.222485,0.799046,0.005371,4.755
801,84a17972cc6d29489bbe205a9e7feb8745726fbc,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7742.667691,100.0,182,110,1460062.0,0.604396,1.654545,774266.769073,5.209486,4.709530,3.593871e+15,"[0.4255338722677228, 0.5361824300106289, 0.620...",0.385478,0.218913,0.797182,0.021403,4.990
802,84b6b2d70924066c8345f2bc2281791ae3188da2,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",6787.268610,100.0,160,82,62797979.0,0.512500,1.951220,678726.860957,5.081404,4.418841,2.122170e+15,"[0.4278805397705206, 0.5370963724059249, 0.620...",0.386378,0.218217,0.796809,0.024609,4.902
803,851d9a4b9b8e236f2d62282ddf06fae57b7d9492,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5277.509215,100.0,389,97,11657734.0,0.249357,4.010309,527750.921516,5.966147,4.584967,7.757397e+14,"[0.43961387728450874, 0.541666084382404, 0.625...",0.390877,0.214824,0.794946,0.040642,6.133


In [37]:
output_df = predict_df[["username", "predicted_score"]]

output_df.to_csv("output_th_new.csv", index=False, header=False)